In [1]:
#pip install opencv-python
import cv2
import numpy as np

# Canny Edge Detection

In [2]:
"""The Canny edge detector is an edge detection operator that uses a multi-stage algorithm to detect a wide range of edges 
in images. It is composed of 5 steps :
1. Noise reduction
2. Gradient calculation
3. Non-maximum suppression
4. Double threshold
5. Edge Tracking by Hysteresis.
"""

'The Canny edge detector is an edge detection operator that uses a multi-stage algorithm to detect a wide range of edges \nin images. It is composed of 5 steps :\n1. Noise reduction\n2. Gradient calculation\n3. Non-maximum suppression\n4. Double threshold\n5. Edge Tracking by Hysteresis.\n'

In [3]:
def gaussian_filter(image,filter_size,sigma):
    filtered_image = cv2.GaussianBlur(image,(filter_size,filter_size),sigma)
    return filtered_image

In [6]:
image = cv2.imread("car_original_image.png")
gray_scale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
gaussian_filter_image = gaussian_filter(gray_scale,5,5)

cv2.imwrite("gaussian_filter_image.jpg", gaussian_filter_image)
#cv2.imshow("gaussian_filter_image", np.hstack((gray_scale, gaussian_filter_image)))
#cv2.waitKey(0)



True

In [7]:
def sobel_filter(image):
    vertical_filter = np.array([[-1,0,1],[-2,0,2],[-1,0,1]])
    horizontal_filter = np.array([[-1,-2,-1],[0,0,0],[1,2,1]])
    
    vertical_filtered = cv2.filter2D(image,-1, vertical_filter)
    horizontal_filtered = cv2.filter2D(image, -1, horizontal_filter)
    
    abs_grad_x = cv2.convertScaleAbs (vertical_filtered)
    abs_grad_y = cv2.convertScaleAbs (horizontal_filtered)
    
    grad = cv2.addWeighted(abs_grad_x,0.5,abs_grad_y,0.5,0 )
    theta = np.rad2deg(np.arctan2(abs_grad_y,abs_grad_x))
    return grad,theta

In [8]:
sobel_filtered, theta = sobel_filter(gaussian_filter_image)
cv2.imwrite("sobel_filtered_image.jpg", sobel_filtered)
#cv2.imshow("sobel_filtered_image", sobel_filtered)

True

In [9]:
def non_max_suppression(image,angles):
    size = image.shape
    print(size)
    suppressed_edges = np.zeros(size)
    for i in range(1,size[0]-1):
        for j in range(1,size[1]-1):
            if (0<=angles[i,j]<22.5) or (157.5<=angles[i,j]<=180) :
                value = max(image[i,j-1] , image[i,j+1])
            elif (22.5<=angles[i,j]<67.5) :
                value = max(image[i-1,j-1] , image[i+1,j+1])
            elif (67.5<=angles[i,j]<112.5) :
                value = max(image[i-1,j] , image[i+1,j])
            else :
                value = max(image[i+1,j-1] , image[i-1,j+1])
                
            if image[i,j] >= value :
                suppressed_edges[i,j] = image[i,j]
        
    suppressed_edges = np.multiply ( suppressed_edges, 255.0/ suppressed_edges.max())
     
    return suppressed_edges
    

In [10]:
nms_matrix = non_max_suppression(sobel_filtered,theta)
cv2.imwrite("non_max_suppression_image.jpg", nms_matrix)

(746, 497)


True

In [11]:
def double_thresholding_hysteresis(image):
    low = 0    #0,50
    high = 50  #20,50,100,150
    weak = 50
    strong = 255
    size = image.shape
    result = np.zeros(size)
    weak_x, weak_y = np.where((image>=low) & (image<high))
    strong_x, strong_y = np.where(image>=high)
    result[weak_x, weak_y] = weak
    result[strong_x, strong_y] = strong
    
    dx = np.array((-1,-1,0,1,1,1,0,-1))
    dy = np.array((0,1,1,1,0,-1,-1,-1))
    while(len(strong_x)):
        x = strong_x[0]
        y = strong_y[0]
        strong_x = np.delete(strong_x,0)
        strong_y = np.delete(strong_y,0)
        for direction in range(len(dx)):
            new_x = x + dx[direction]
            new_y = y + dy[direction]
            
        if ((new_x >= 0  & new_x < size[0]) & (new_y >= 0  & new_y < size[1]) and (result[new_x,new_y] == weak)):
            result[new_x,new_y] = strong
            np.append(strong_x, new_x)
            np.append(strong_y, new_y)
    result [result != strong] = 0
    return result   

In [12]:
double_thresholding_hysteresis = double_thresholding_hysteresis(nms_matrix)
cv2.imwrite("Final_image.jpg", double_thresholding_hysteresis)

True

In [13]:
""" By reducing  the upper threshold value which is used to identify weak & strong pixels, more edges are detected.
but by increasing the upper threshold value, only the edges which are strong and connected are identified.
So we must tune the values of threshold as per our input image and application"""

' By reducing  the upper threshold value which is used to identify weak & strong pixels, more edges are detected.\nbut by increasing the upper threshold value, only very strong edges which are strong and connected are identified.\nSo we must tune the values of threshold as per our input image and application'